SN Photometric Challenge Parser

### First, an example of SNANASims dealing with SNANA FITS file

In [ ]:
import sndata as snd
from sndata import SNANASims
from astropy.table import Table
from matplotlib import pyplot as plt
megacamBandNames = 'ugriz'
#megacamRegisteredNames = tuple('megacam' + band for band in megacamBandNames)
megacamRegisteredNames = tuple('megacam' + band for band in megacamBandNames)

In [ ]:
snanafile = SNANASims.fromSNANAfileroot(snanafileroot='snana_fits',
                                       location='../sndata/example_data/',
                                       coerce_inds2int=False,
                                       SNANABandNames=megacamBandNames,
                                       registeredBandNames=megacamRegisteredNames)

In [ ]:
#print(snanafile.photFile)
#print(snanafile.headFile)
#snanafile.headData
print(snanafile.phot)
#print(snanafile.bandNames)
#print(snanafile.newbandNames)
#print(snanafile.bandNameDict)


In [ ]:
lcInstance = snanafile.get_SNANA_photometry(ptrs=[0,40])


In [ ]:
lcInstance.lightCurve[['mjd', 'band', 'flux', 'fluxerr', 'zp', 'zpsys']]


In [ ]:
lcInstance.bandNameDict

In [ ]:
snanalc = snd.LightCurve(lcInstance.lightCurve[['mjd', 'band', 'flux', 'fluxerr', 'zp', 'zpsys']])


In [ ]:
snanalc

### Second, check if SNANASims is able to read old SNANA ASCII files from SN Challenge 2010

In [ ]:
import sndata as snd
from sndata import SNChalSims
from astropy.table import Table

megacamBandNames = 'ugriz'
#megacamRegisteredNames = tuple('megacam' + band for band in megacamBandNames)
megacamRegisteredNames = tuple('megacam' + band for band in megacamBandNames)

In [ ]:
charfile = SNChalSims.fromSNChal(snanafileroot='DES_BLIND+HOSTZ.tar.gz', zipped=True,
                                       location='../sndata/example_data/',
                                       SNANABandNames=megacamBandNames,
                                       registeredBandNames=megacamRegisteredNames)
#charfile = SNChalSims.fromSNChal(snanafileroot='DES_SN000041.DAT',zipped=False,
#                                       location='../sndata/example_data/',
#                                       SNANABandNames=megacamBandNames,
#                                       registeredBandNames=megacamRegisteredNames)

In [ ]:
charfile.phot
#print(charfile.photFile)
#print(charfile.headFile)
#print(charfile.phot)
#print(charfile.bandNames)
#print(charfile.newbandNames)
#print(charfile.bandNameDict)

In [ ]:
lcInstance = charfile.get_SNANA_photometry(snid=41)

In [ ]:
lcInstance

In [ ]:
t=lightcurve[lightcurve.FLT =='i']
plt.errorbar(lightcurve[lightcurve.FLT =='g']['MJD'],lightcurve[lightcurve.FLT =='g']['MAG'],yerr=lightcurve[lightcurve.FLT =='g']['MAGERR'], capsize=0,fmt='--o')
plt.errorbar(lightcurve[lightcurve.FLT =='r']['MJD'],lightcurve[lightcurve.FLT =='r']['MAG'],yerr=lightcurve[lightcurve.FLT =='r']['MAGERR'], capsize=0,fmt='--o')
plt.errorbar(lightcurve[lightcurve.FLT =='i']['MJD'],lightcurve[lightcurve.FLT =='i']['MAG'],yerr=lightcurve[lightcurve.FLT =='i']['MAGERR'], capsize=0,fmt='--o')
plt.errorbar(lightcurve[lightcurve.FLT =='z']['MJD'],lightcurve[lightcurve.FLT =='z']['MAG'],yerr=lightcurve[lightcurve.FLT =='z']['MAGERR'], capsize=0,fmt='--o')
plt.gca().invert_yaxis()
plt.show()